In [1]:
import sys
import os
sys.path.append('..')

import json
import numpy as np
import scipy.spatial
import util

In [2]:
# key definitions
key_notes = {
    'C': ['C', 'D', 'E', 'F', 'G', 'A', 'B'],
    'C#': ['C#', 'D#', 'F', 'F#', 'G#', 'A#', 'C'],
    'D': ['D', 'E', 'F#', 'G', 'A', 'B', 'C#'],
    'D#': ['D#', 'F', 'G', 'G#', 'A#', 'C', 'D'],
    'E': ['E', 'F#', 'G#', 'A', 'B', 'C#', 'D#'],
    'F': ['F', 'G', 'A', 'A#', 'C', 'D', 'E'],
    'F#': ['F#', 'G#', 'A#', 'B', 'C#', 'D#', 'F'],
    'G': ['G', 'A', 'B', 'C', 'D', 'E', 'F#'],
    'G#': ['G#', 'A#', 'C', 'C#', 'D#', 'F', 'G'],
    'A': ['A', 'B', 'C#', 'D', 'E', 'F#', 'G#'],
    'A#': ['A#', 'C', 'D', 'D#', 'F', 'G', 'A'],
    'B': ['B', 'C#', 'D#', 'E', 'F#', 'G#', 'A#'],
    'Cm': ['D#', 'F', 'G', 'G#', 'A#', 'C', 'D'],
    'C#m':['E', 'F#', 'G#', 'A', 'B', 'C#', 'D#'],
    'Dm': ['F', 'G', 'A', 'A#', 'C', 'D', 'E'],
    'D#m':['F#', 'G#', 'A#', 'B', 'C#', 'D#', 'F'],
    'Em': ['G', 'A', 'B', 'C', 'D', 'E', 'F#'],
    'Fm': ['G#', 'A#', 'C', 'C#', 'D#', 'F', 'G'],
    'F#m':['A', 'B', 'C#', 'D', 'E', 'F#', 'G#'],
    'Gm': ['A#', 'C', 'D', 'D#', 'F', 'G', 'A'],
    'G#m':['B', 'C#', 'D#', 'E', 'F#', 'G#', 'A#'],
    'Am': ['C', 'D', 'E', 'F', 'G', 'A', 'B'],
    'A#m': ['C#', 'D#', 'F', 'F#', 'G#', 'A#', 'C'],
    'Bm':['D', 'E', 'F#', 'G', 'A', 'B', 'C#']
}

for k in key_notes:
    if k.endswith('m'):
        key_notes[k] = [x+y for x,y in zip(key_notes[k], ['m', 'dim', '', 'm', 'm', '', ''])]
    else:
        key_notes[k] = [x+y for x,y in zip(key_notes[k], ['', 'm', 'm', '', '', 'm', 'dim'])]

key_indices = { x:i for i,x in enumerate(key_notes.keys()) }

key_vectors = { x:np.zeros(len(key_indices)) for x in key_indices }
for k in key_vectors:
    for chord in key_notes[k]:
        if not 'dim' in chord:
            key_vectors[k][key_indices[chord]] = 1.0

In [86]:
with open('../data/processed_sheets/Alton Delmore and Rabon Delmore - Blues Stay Away From Me (Transcribed).json') as f:
    sheet = json.load(f)

In [95]:
sheet_hist = {x:0 for x in key_notes}

for s in sheet:
    chord = s['chord']['root']
    alter = s['chord']['alter']
    chord = chord + alter*'#'
    mode = s['chord']['mode']
    if mode == 'MINOR':
        chord += 'm'
    elif mode == 'DIMINISHED':
        chord += 'dim'
    
    if 'dim' not in chord:
        sheet_hist[chord] += 1
    
sheet_vec = np.zeros(len(key_indices))
for k in sheet_hist:
    sheet_vec[key_indices[k]] = sheet_hist[k]
    
distances = list(map(lambda k:(k,scipy.spatial.distance.cosine(sheet_vec, key_vectors[k])), key_vectors.keys()))
distances = sorted(distances, key=lambda e:e[1])
predicted_key = distances[0][0]
print(predicted_key)

A#


In [4]:
predicted_keys = {x:0 for x in key_notes}

for psheet in os.listdir('../data/processed_sheets/'):
    if psheet.endswith('.json') and not psheet.startswith('.'):
        with open(os.path.join('../data/processed_sheets/', psheet)) as f:
            sheet = json.load(f)
            sheet_hist = {x:0 for x in key_notes}
            
            for s in sheet:
                chord = s['chord']['root']
                alter = s['chord']['alter']
                chord = chord + alter*'#'
                mode = s['chord']['mode']
                if mode == 'MINOR':
                    chord += 'm'
                elif mode == 'DIMINISHED':
                    chord += 'dim'

                if 'dim' not in chord:
                    sheet_hist[chord] += 1

            sheet_vec = np.zeros(len(key_indices))
            for k in sheet_hist:
                sheet_vec[key_indices[k]] = sheet_hist[k]

            distances = list(map(lambda k:(k,scipy.spatial.distance.cosine(sheet_vec, key_vectors[k])), key_vectors.keys()))
            distances = sorted(distances, key=lambda e:e[1])
            predicted_key = distances[0][0]
            predicted_keys[predicted_key] += 1

/usr/lib/python3/dist-packages/scipy/spatial/distance.py:505: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - np.dot(u, v) / (norm(u) * norm(v))


In [5]:
predicted_keys

{'A': 121,
 'A#': 528,
 'A#m': 6,
 'Am': 9,
 'B': 19,
 'Bm': 3,
 'C': 1703,
 'C#': 44,
 'C#m': 2,
 'Cm': 12,
 'D': 371,
 'D#': 665,
 'D#m': 4,
 'Dm': 22,
 'E': 92,
 'Em': 13,
 'F': 1059,
 'F#': 20,
 'F#m': 9,
 'Fm': 13,
 'G': 875,
 'G#': 148,
 'G#m': 3,
 'Gm': 22}